En este notebook se crea una estrategia de medias moviles de trading para el S&P500

In [1]:
# Importar las librerias necesarias
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
import cufflinks as cf
cf.set_config_file(offline=True)

In [2]:
# Bajar datos de IBM, utilizando fecha de inicio y fin
data = yf.download('SPY', start='2000-01-01' , end='2024-11-08', progress=False)
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2000-01-03 00:00:00+00:00,93.004906,145.437500,148.250000,143.875000,148.250000,8164300
2000-01-04 00:00:00+00:00,89.367821,139.750000,144.062500,139.640625,143.531250,8089800
2000-01-05 00:00:00+00:00,89.527718,140.000000,141.531250,137.250000,139.937500,12177900
2000-01-06 00:00:00+00:00,88.088875,137.750000,141.500000,137.750000,139.625000,6227200
2000-01-07 00:00:00+00:00,93.204758,145.750000,145.750000,140.062500,140.312500,8066500
...,...,...,...,...,...,...
2024-11-01 00:00:00+00:00,571.039978,571.039978,575.549988,570.619995,571.320007,45667500
2024-11-04 00:00:00+00:00,569.809998,569.809998,572.500000,567.890015,571.179993,38217000


In [3]:
# Dividir los datos en conjuntos de entrenamiento (in sample) y de prueba (out sample)
insample = int(len(data)*0.8)
outsample = len(data) - insample

df = data[:insample]
df.tail()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2019-11-12 00:00:00+00:00,286.039368,309.000000,309.989990,308.149994,308.750000,46484600
2019-11-13 00:00:00+00:00,286.131866,309.100006,309.540009,307.660004,307.910004,53917700
2019-11-14 00:00:00+00:00,286.548401,309.549988,309.640015,308.089996,308.790009,51219900
2019-11-15 00:00:00+00:00,288.622009,311.790009,311.839996,310.260010,311.019989,62023600
2019-11-18 00:00:00+00:00,288.834869,312.019989,312.279999,311.029999,311.529999,49228000


In [4]:
insample

5002

In [5]:
outsample

1251

In [6]:
# Definir los parámetros de las medias móviles
MA1 = 50 # Media móvil más corta
MA2 = 200 # Media móvil más larga

In [7]:
# Calcula las medias móviles usando Pandas
df['SMA'] = df['Adj Close'].rolling(MA1).mean()
df['LMA'] = df['Adj Close'].rolling(MA2).mean()

df.dropna(inplace=True)

In [8]:
df.tail()

Price,Adj Close,Close,High,Low,Open,Volume,SMA,LMA
Ticker,SPY,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,,
2019-11-12 00:00:00+00:00,286.039368,309.000000,309.989990,308.149994,308.750000,46484600,276.794573,266.640264
2019-11-13 00:00:00+00:00,286.131866,309.100006,309.540009,307.660004,307.910004,53917700,277.098415,266.849478
2019-11-14 00:00:00+00:00,286.548401,309.549988,309.640015,308.089996,308.790009,51219900,277.340927,267.050048
2019-11-15 00:00:00+00:00,288.622009,311.790009,311.839996,310.260010,311.019989,62023600,277.620673,267.260392
2019-11-18 00:00:00+00:00,288.834869,312.019989,312.279999,311.029999,311.529999,49228000,277.901912,267.463127


In [9]:
df[['Adj Close','SMA','LMA']].iplot(title='Activo y Medias Móviles')

In [10]:
df['SIGNAL'] = np.where(df['SMA'] > df['LMA'], 1, 0) # Agrega la col Signal poniendo 1 o 0 según la regla de trading
df['SIGNAL'] = df['SIGNAL'].shift(1)                 # Asume que la posición se toma al dia siguiente de la señal

In [11]:
# Calcular los rendimientos de la estrategia
df['ASSET_RETURN'] = np.log(df['Adj Close']).diff().fillna(0)
df['STRATEGY'] = (df['SIGNAL'] * df['ASSET_RETURN']).fillna(0)

In [12]:
np.exp(df[['ASSET_RETURN', 'STRATEGY']].sum())

,,0
Price,Ticker,
ASSET_RETURN,,3.244426
STRATEGY,,4.316285


In [13]:
df[['STRATEGY', 'ASSET_RETURN']].cumsum().apply(np.exp).iplot(title='Performance de la Estrategia versus Buy and Hold')

In [14]:
# Calcular CAGR

def cagr(returns):

    cumulative_returns = np.exp(returns.sum())    # retornos contínuos

    years = len(returns)/252

    return (cumulative_returns)**(1/years) - 1

In [15]:
cagr(df['STRATEGY'])

0.07974811602524312

In [16]:
cagr(df['ASSET_RETURN'])

0.06369710715644761

In [17]:
# Calcular la volatilidad

def volatility(returns):
    return returns.std()*np.sqrt(252)

In [18]:
volatility(df['STRATEGY'])

0.11371201694192151

In [19]:
volatility(df['ASSET_RETURN'])

0.18809269152705074

In [20]:
# Calcular Sharpe Ratio
def sharpe(returns, rf=0.0):
    return (cagr(returns) - rf) / volatility(returns)

In [21]:
# Calcular Max DD

def maxdd(returns):
    cumulative_returns = np.exp(returns.cumsum())
    drawdown_percentage = (cumulative_returns.cummax() - cumulative_returns) / cumulative_returns.cummax()
    return drawdown_percentage.max()

In [22]:
maxdd(df['STRATEGY'])

0.19178966774663211

In [23]:
# Calcular Calmar Ratio
def calmar(returns):
    return cagr(returns)/maxdd(returns)

In [24]:
calmar(df['STRATEGY'])

0.41581028301585093

In [25]:
stats = {'Fecha inicio': df.index[0].date(),
            'Fecha fin': df.index[-1].date(),
            'Total Returns %': np.round((np.exp(df['STRATEGY'].sum())-1)*100,2),
            'CAGR %': np.round(cagr(df['STRATEGY'])*100,2),
            'Vol anual %': np.round(volatility(df['STRATEGY'])*100,2),
            'Sharpe Ratio': np.round(sharpe(df['STRATEGY']),2),
            'Max DD %': np.round(maxdd(df['STRATEGY'])*100,2),
            'Calmar Ratio': np.round(calmar(df['STRATEGY']),2),
            'Correlación': np.round(df[['ASSET_RETURN']].corrwith(df['STRATEGY'])[0],2) }

In [26]:
stats

{'Fecha inicio': datetime.date(2000, 10, 16),
 'Fecha fin': datetime.date(2019, 11, 18),
 'Total Returns %': 331.63,
 'CAGR %': 7.97,
 'Vol anual %': 11.37,
 'Sharpe Ratio': 0.7,
 'Max DD %': 19.18,
 'Calmar Ratio': 0.42,
 'Correlación': 0.6}

In [27]:
pd.DataFrame(stats, index=['Estrategia']).T

,Estrategia
Fecha inicio,2000-10-16
Fecha fin,2019-11-18
Total Returns %,331.63
CAGR %,7.97
Vol anual %,11.37
Sharpe Ratio,0.7
Max DD %,19.18
Calmar Ratio,0.42
Correlación,0.6


In [28]:
mom = df[['STRATEGY']].resample('M').sum()
mom['Year'] = mom.index.year
mom['Month'] = mom.index.month
mom

Price,STRATEGY,Year,Month
Ticker,,,
Date,,,
2000-10-31 00:00:00+00:00,0.033905,2000,10
2000-11-30 00:00:00+00:00,-0.001750,2000,11
2000-12-31 00:00:00+00:00,0.000000,2000,12
2001-01-31 00:00:00+00:00,0.000000,2001,1
2001-02-28 00:00:00+00:00,0.000000,2001,2
...,...,...,...
2019-07-31 00:00:00+00:00,0.015006,2019,7
2019-08-31 00:00:00+00:00,-0.016885,2019,8


In [29]:
# make pivot table
mom = mom.pivot(index='Year', columns='Month').fillna(0)
mom

Price   STRATEGY                                                              \
Ticker                                                                         
Month         1         2         3         4         5         6         7    
Year                                                                           
2000    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2001    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2002    0.000000  0.000000  0.000000 -0.045491  0.027795  0.000000  0.000000   
2003    0.000000  0.000000  0.000000  0.000000  0.021582  0.010595  0.017867   
2004    0.019577  0.013479 -0.013334 -0.019102  0.016978  0.018333 -0.032749   
2005   -0.022676  0.020688 -0.018462 -0.018912  0.031716  0.001514  0.037548   
2006    0.023730  0.005709  0.016369  0.012553 -0.030584  0.002605 -0.004883   
2007    0.014928 -0.019812  0.011522  0.043343  0.033358 -0.014729 -0.031811   
2008    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2009    0.000000  0.000000  0.000000  0.000000  0.000000  0.002700  0.071954   
2010   -0.037019  0.030718  0.059098  0.015352 -0.082789 -0.053127  0.027613   
2011    0.023033  0.034147  0.000120  0.028550 -0.011278 -0.017014 -0.020208   
2012   -0.002206  0.042489  0.031657 -0.006698 -0.061935  0.039780  0.011760   
2013    0.049923  0.012678  0.037268  0.019030  0.023335 -0.013434  0.050386   
2014   -0.035885  0.044511  0.008261  0.006927  0.022942  0.020434 -0.013529   
2015   -0.030077  0.054682 -0.015830  0.009786  0.012774 -0.020522  0.022338   
2016   -0.082052  0.000000  0.000000 -0.018107  0.016869  0.003470  0.035822   
2017    0.017736  0.038539  0.001249  0.009877  0.014014  0.006355  0.020346   
2018    0.054828 -0.037038 -0.027793  0.005155  0.024018  0.005734  0.036377   
2019    0.000000  0.000000  0.004826  0.040040 -0.065895  0.067272  0.015006   

Price                                                     
Ticker                                                    
Month         8         9         10        11        12  
Year                                                      
2000    0.000000  0.000000  0.033905 -0.001750  0.000000  
2001    0.000000  0.000000  0.000000  0.000000  0.000000  
2002    0.000000  0.000000  0.000000  0.000000  0.000000  
2003    0.020415 -0.010966  0.052143  0.010862  0.049088  
2004    0.002343  0.000000  0.002831  0.043555  0.029676  
2005   -0.009419  0.007993 -0.023935  0.043014 -0.001915  
2006    0.000459  0.026643  0.031030  0.019690  0.013283  
2007    0.012751  0.037984  0.013475 -0.039503 -0.003897  
2008    0.000000  0.000000  0.000000  0.000000  0.000000  
2009    0.036273  0.034843 -0.019413  0.059784  0.018921  
2010    0.000000  0.000000  0.004822  0.000000  0.064713  
2011   -0.077629  0.000000  0.000000  0.000000  0.000000  
2012    0.024744  0.025035 -0.018366  0.005643  0.008895  
2013   -0.030452  0.031157  0.045266  0.029207  0.025596  
2014    0.038705 -0.013892  0.023278  0.027101 -0.002540  
2015   -0.062887 -0.010783  0.000000  0.000000 -0.001251  
2016    0.001197  0.000058 -0.017489  0.036176  0.020069  
2017    0.002913  0.019949  0.023291  0.030108  0.012055  
2018    0.031421  0.005928 -0.071608  0.018379 -0.037667  
2019   -0.016885  0.019270  0.021864  0.028246  0.000000

In [30]:
mom.columns = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
mom = np.round(mom*100, 2)
mom

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic
Year,,,,,,,,,,,,
2000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.39,-0.18,0.00
2001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2002,0.00,0.00,0.00,-4.55,2.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2003,0.00,0.00,0.00,0.00,2.16,1.06,1.79,2.04,-1.10,5.21,1.09,4.91
2004,1.96,1.35,-1.33,-1.91,1.70,1.83,-3.27,0.23,0.00,0.28,4.36,2.97
2005,-2.27,2.07,-1.85,-1.89,3.17,0.15,3.75,-0.94,0.80,-2.39,4.30,-0.19
2006,2.37,0.57,1.64,1.26,-3.06,0.26,-0.49,0.05,2.66,3.10,1.97,1.33
2007,1.49,-1.98,1.15,4.33,3.34,-1.47,-3.18,1.28,3.80,1.35,-3.95,-0.39
2008,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [31]:
mom.iplot(kind='heatmap', colorscale='RdYlGn', title='Rendimientos mensuales')

In [32]:
yoy = pd.DataFrame(df[['ASSET_RETURN','STRATEGY']].resample('A').sum())
yoy['Year'] = yoy.index.year
yoy.set_index('Year', inplace=True)
yoy = yoy*100
yoy.iplot(kind='bar', title ='Estrategia Vs Buy and Hold: Rendimientos YoYear (%)')

In [33]:
# Distribución de los rendimientos de la estrategia (considera solo no nulos)
df[df['STRATEGY']!=0]['STRATEGY'].iplot(kind='histogram', title = 'Distribución de los rendimientos de la estrategia')

In [34]:
def MA_opt(precio, ma1, ma2):
    opt = pd.DataFrame()

    #calcula las medias móviles
    opt['sma'] = precio.rolling(ma1).mean()
    opt['lma'] = precio.rolling(ma2).mean()

    #genera señal
    opt['signal'] = np.where(opt['sma'] > opt['lma'], 1, 0)
    opt['signal'] = opt['signal'].shift(1)

    #calcula los retornos de la estrategia
    opt['retornos'] = np.log(precio).diff().fillna(0)
    opt['strategia'] = (opt['signal'].shift(1) * opt['retornos']).fillna(0)

    return sharpe(opt['strategia'])

In [ ]:
#itera para distintas combinaciones de ventanas

resultados = []
ma1 = []
ma2 = []

for s in range(5,55,5):
    for l in range(55,255,5):
        ma1.append(s)
        ma2.append(l)
        resultados.append(MA_opt(df['Adj Close'],s,l))

output = pd.DataFrame([ma1, ma2, resultados]).T
output.columns = ['sma','lma','sharpe_ratio']

In [ ]:
#valor máximo de Sharpe y número de índice
output['sharpe_ratio'].max(), output['sharpe_ratio'].idxmax()

In [ ]:
output[105:108]

In [ ]:
#máx y min Sharpe
output['sharpe_ratio'].min(), output['sharpe_ratio'].max()

In [ ]:
print("Máximo Sharpe Ratio igual a {0}, para la combinación {1} y {2} de ventanas.".
      format(round(output['sharpe_ratio'].max(),2),
             output['sma'][output['sharpe_ratio'].idxmax()],
             output['lma'][output['sharpe_ratio'].idxmax()]))

In [ ]:
df_os = df = data[outsample:]
df_os.tail()

In [ ]:
#calcular las medias móviles
df_os['SMA'] = df_os['Adj Close'].rolling(15).mean()
df_os['LMA'] = df_os['Adj Close'].rolling(185).mean()

df_os['SIGNAL'] = np.where(df_os['SMA'] > df_os['LMA'], 1, 0) # Agrega la col Signal poniendo 1 o 0 según la regla de trading
df_os['SIGNAL'] = df_os['SIGNAL'].shift(1)

# Calcular los rendimientos de la estrategia
df_os['ASSET_RETURN'] = np.log(df_os['Adj Close']).diff().fillna(0)
df_os['STRATEGY'] = (df_os['SIGNAL'] * df_os['ASSET_RETURN']).fillna(0)

In [ ]:
df_os[['STRATEGY', 'ASSET_RETURN']].cumsum().apply(np.exp).iplot(title=
                                                                 'Performance de la Estrategia Outsample versus Buy and Hold')

In [ ]:
np.exp(df_os[['ASSET_RETURN', 'STRATEGY']].sum())

In [ ]:
stats_os = {'Fecha inicio': df_os.index[0].date(),
            'Fecha fin': df_os.index[-1].date(),
            'Total Returns %': np.round((np.exp(df_os['STRATEGY'].sum())-1)*100,2),
            'CAGR %': np.round(cagr(df_os['STRATEGY'])*100,2),
            'Vol anual %': np.round(volatility(df_os['STRATEGY'])*100,2),
            'Sharpe Ratio': np.round(sharpe(df_os['STRATEGY']),2),
            'Max DD %': np.round(maxdd(df_os['STRATEGY'])*100,2),
            'Calmar Ratio': np.round(calmar(df_os['STRATEGY']),2),
            'Correlación': np.round(df_os[['ASSET_RETURN']].corrwith(df_os['STRATEGY'])[0],2) }

In [ ]:
pd.DataFrame(stats_os, index=['Estrategia Outsample']).T